In [6]:
from itertools import starmap
from pathlib import Path
from typing import Tuple

from X import X
from algebra import Matrix, Vector, vector
from bayes import NaiveBayes
from logistic_regression import LogisticRegression


def load(filepath: Path) -> Tuple[Matrix, Vector]:
    assert filepath.exists() and filepath.is_file()

    with filepath.open() as file:
        lines = file.readlines()

    samples = map(lambda l: l.split(), lines)
    samples = (tuple(map(lambda v: int(v), s)) for s in samples)
    samples = ((s[:-1], s[-1]) for s in samples)

    return tuple(zip(*samples))

In [7]:
x,y = load(Path('breast-cancer.data'))
x = X(vector(map(lambda xi: vector(map(lambda xij: xij-1, xi)), x)))
y = vector(map(lambda r: 0 if r == 2 else 1, y))

In [8]:
def compute_error(pred: Vector, y: Vector) -> float:
    assert len(pred) == len(y)
    same = sum(starmap(lambda a, b: a == b, zip(pred, y)))
    return 1. - (float(same) / len(y))


In [3]:
b = NaiveBayes(9,10)
h = b.train(x, y)

In [4]:
lr = LogisticRegression()
h2 = lr.train(x, y)

1.0
1.0
